In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv('/kaggle/input/sf-crime/train.csv.zip')
train_df.head()

In [ ]:
test_df = pd.read_csv('/kaggle/input/sf-crime/test.csv.zip')
test_df.head()

In [ ]:
train_df.info()

In [ ]:
test_df.info()

In [ ]:
train_df.drop(['Descript','Resolution','Address'],inplace=True,axis=1)
test_df.drop('Address',inplace =True, axis = 1)

In [ ]:
train_df['DayOfWeek'].replace(to_replace = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'], value = [i for i in range(7)],inplace=True)
test_df['DayOfWeek'].replace(to_replace = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'], value = [i for i in range(7)],inplace=True)

In [ ]:
train_dummies = pd.get_dummies(train_df['PdDistrict'])
test_dummies = pd.get_dummies(test_df['PdDistrict'])

train_df = pd.concat([train_df,train_dummies], axis=1)
train_df.drop('PdDistrict',inplace =True, axis=1)

test_df = pd.concat([test_df,test_dummies], axis=1)
test_df.drop('PdDistrict',inplace =True, axis=1)


In [ ]:
train_df.head()

In [ ]:
l1 = list(train_df['Dates'])
for i in range(len(l1)):
    l1[i] = l1[i][:4:]
train_df['Year']=l1

l1 = list(train_df['Dates'])
for i in range(len(l1)):
    l1[i] = l1[i][5:7:]
train_df['Month']=l1

l1 = list(train_df['Dates'])
for i in range(len(l1)):
    l1[i] = l1[i][8:10:]
train_df['Day']=l1

l1 = list(train_df['Dates'])
for i in range(len(l1)):
    l1[i] = l1[i][11:-6:]
train_df['Hours']=l1

l1 = list(train_df['Dates'])
for i in range(len(l1)):
    l1[i] = l1[i][-5:-3:]
train_df['Minutes']=l1



In [ ]:
l1 = list(test_df['Dates'])
for i in range(len(l1)):
    l1[i] = l1[i][:4:]
test_df['Year']=l1

l1 = list(test_df['Dates'])
for i in range(len(l1)):
    l1[i] = l1[i][5:7:]
test_df['Month']=l1

l1 = list(test_df['Dates'])
for i in range(len(l1)):
    l1[i] = l1[i][8:10:]
test_df['Day']=l1

l1 = list(test_df['Dates'])
for i in range(len(l1)):
    l1[i] = l1[i][11:-6:]
test_df['Hours']=l1

l1 = list(test_df['Dates'])
for i in range(len(l1)):
    l1[i] = l1[i][-5:-3:]
test_df['Minutes']=l1


In [ ]:
train_df.drop('Dates',axis=1,inplace=True)
test_df.drop('Dates',axis=1,inplace=True)

In [ ]:
test_df.head()

In [ ]:
X = train_df.drop(['Category'],axis=1)
y = pd.get_dummies(train_df['Category'])

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
X = X.astype(float)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1,random_state=42)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

In [ ]:
from tensorflow import keras
from keras.layers import Dense,Activation
from keras.models import Sequential

In [ ]:
model = Sequential()
model.add(Dense(100, input_shape=(X.shape[1],)))
model.add(Activation('relu'))
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dense(80))
model.add(Activation('relu'))
model.add(Dense(60))
model.add(Activation('relu'))
model.add(Dense(39))
model.add(Activation('softmax'))
model.summary()

In [ ]:
model.compile(optimizer ='adam',
             loss = 'categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
r = model.fit(X_train,y_train, 
              batch_size = 32, 
              epochs = 20, 
              verbose = 2, 
              validation_data=(X_test,y_test))

In [ ]:
test = test_df.drop(['Id'],axis=1)
test = test.astype(float)

In [ ]:
prediction = model.predict(test)

In [ ]:
s = np.max(prediction, axis =1).reshape(-1,1)
final = np.array((prediction == s), dtype = 'int32')

In [ ]:
result = pd.DataFrame()
result['Id'] = test_df['Id']

In [ ]:
sample=pd.read_csv('/kaggle/input/sf-crime/sampleSubmission.csv.zip')
column_names = list(sample.columns)
column_names.remove('Id')
for i, e in enumerate(column_names):
    result[e] = final[:,i]

In [ ]:
result.head()

In [ ]:
result.to_csv('Submission.csv',index=False)